# Feature Engineering - add discovered knowledge

In this notebook I add the knowledge discovered on the web to the dataframes.

I have also created groups of breads, but it doesn't work very well. Thus, I don't add this feature but the code is still here

In [3]:
from __future__ import division
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import matplotlib 
import numpy as np
# Enable inline plotting
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD

from sklearn.manifold import TSNE

from sklearn import preprocessing

from urllib import urlopen
from bs4 import BeautifulSoup
import nltk 

from dateutil import parser

import pyprind

from difflib import SequenceMatcher

import seaborn as sns
sns.set(style="darkgrid", palette="muted")
import matplotlib
matplotlib.style.use('ggplot')
pd.set_option('display.notebook_repr_html', True)
plt.rcParams['figure.figsize'] = 10, 6

import warnings
warnings.filterwarnings('ignore')

/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1035: UserWarning: Duplicate key in file "/Users/valentin/anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/matplotlibrc", line #513
  (fname, cnt))


In [4]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train[:3]

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White


In [5]:
import csv
with open('breed2score.csv', mode='r') as infile:
    reader = csv.reader(infile)
    mydict = {rows[0]:rows[1] for rows in reader}
      
import pickle
breed2score = pickle.load( open( "breed2score.p", "rb" ) )


In [6]:
# a simple sequence matcher function
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

all_breed_info = list(set(list(df_train['Breed']) + list(df_test['Breed'])))
unique_breeds = set()
for i in all_breed_info:
    i = i.replace(' Mix' , '')
    for ii in i.split('/'):
        unique_breeds.add(ii)
print len(unique_breeds)

def matchBreedScore( BreedElement ):
    c = sorted( breed2score.items() , key = lambda x: similar(x[0], BreedElement) , reverse = True )[0]
    sim =  similar(c[0] , BreedElement)
    if sim < 0.7:
        sim = 0
    return sim

dataBreed2score = { e : matchBreedScore(e) for e in unique_breeds}

def BreedInfo2Score( e ):
    e = e.replace(' Mix' , '')
    sims = []
    for ee in e.split('/'):
        sims.append( dataBreed2score[ee] )
    return np.mean(sims)

237


In [7]:
# retrieve dog breed groups from http://www.akc.org/dog-breeds/ 
# NB I have not used these features for the final submission
group_names = ['herding' , 'hound', 'non-sporting', 'sporting', 'terrier', 'toy', 'working']
dog_groups = dict()
for g in group_names:
    dog_groups[g] = []
root = "http://www.akc.org/dog-breeds/groups/"    
for g in group_names:
    url = root + g + '/'
    print "reading from UR: " + url
    html = urlopen(url).read()    
    soup = BeautifulSoup ( html )
    h2s = soup.findAll('h2')

    for h2 in h2s:
        for link in h2.findAll('a'):
            dog_groups[g].append(link.contents[0])
group_names.append('Unknown')
dog_groups['Unknown'] = ['Unknown']
dog_groups['Pit Bull'] = ['Pit Bull']
# define cats groups thanks to http://www.acf.asn.au/index.php?page=standards
cat_groups = dict()
a = [ 'Aphrodite','Birman','Exotic','Maine Coon','Neva Masquerade','Norwegian Forest Cat', 'Persian', 'Ragdoll', 'Siberian','Turkish Van']
b = ['Balinese Foreign White', 'Oriental Shorthair', 'Oriental Longhair', 'Javanese', 'Peterbald','Siamese' ]
c = ['Abyssinian','American Curl','American Shorthair', 'Australian Mist', 'Australian Mist','Bengal', 'Bombay (American)', 'British Shorthair',\
                  'Burmese', 'Burmese (American)','Burmilla' , 'Cornish Rex','Cymric','Devon Rex','Egyptian Mau','Japanese Bobtail',\
                  'Korat','La Perm ', 'Mandalay','Manx','Ocicat','Pixiebob','Russian','Scottish Fold','Scottish Shorthair',\
                  'Selkirk Rex','Singapura','Snowshoe','Somali','Sphynx','Tonkinese']
d = ['Domestic shortair']
e = ['Domestic Medium Hair']
f = ['Domestic Longhair']

cat_groups['cat1'] = a+b+c
cat_groups['cat2'] = d
cat_groups['cat3'] = e
cat_groups['cat4'] = f

all_groups = cat_groups.copy()
all_groups.update(dog_groups)

reading from UR: http://www.akc.org/dog-breeds/groups/herding/
reading from UR: http://www.akc.org/dog-breeds/groups/hound/
reading from UR: http://www.akc.org/dog-breeds/groups/non-sporting/
reading from UR: http://www.akc.org/dog-breeds/groups/sporting/
reading from UR: http://www.akc.org/dog-breeds/groups/terrier/
reading from UR: http://www.akc.org/dog-breeds/groups/toy/
reading from UR: http://www.akc.org/dog-breeds/groups/working/


In [8]:
breed2group = dict()
for b in unique_breeds:        
    sim = 0
    g_final = 'Unknown'
    for g in all_groups:
        if sim < 1: # perfect match
            for gg in all_groups[g]:
                s = similar( gg , b)
                if s > 0.6 and s > sim:
                    sim = s
                    g_final = g
    breed2group[b] = g_final 

In [9]:
def BreedData2Groups(df):
    for k in all_groups.keys():
        df['group='+k] = [False for x in range(len(df))]
    for b in df['Breed']:
        b = b.replace(' Mix' , '')
        i = 0
        for bb in b.split('/'):
            group = breed2group[bb]
            df.loc[i, 'group='+ group] = True
            i +=1
    return df

In [10]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
#df_train = BreedData2Groups(df_train)
df_train['BreedScore'] = df_train['Breed'].map(BreedInfo2Score)
#df_test = BreedData2Groups(df_test)
df_test['BreedScore'] = df_test['Breed'].map(BreedInfo2Score)
df_train.to_csv('train_breed_infos.csv')
df_test.to_csv('test_breed_infos.csv')